# 1.4.2 RCTデータの集計と有意差検定

In [35]:
import random

import numpy as np
import pandas as pd

## データの準備

In [3]:
data = pd.read_csv("data_cibook.csv")
data.shape

(42613, 13)

In [4]:
data.head(3)

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,treatment
0,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0,0
1,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0,1
2,9,5) $500 - $750,675.07,1,1,Rural,1,Phone,Mens E-Mail,0,0,0.0,1


## 集計

In [5]:
# 介入群とコントロール群での、購入の発生確率と購入額の平均
data.groupby("treatment")[["conversion", "spend"]].mean()

,conversion,spend
treatment,,
0,0.005726,0.652789
1,0.012531,1.422617


## 有意差検定

In [6]:
from scipy import stats

### t検定

In [8]:
# 介入群 (男性向けメールを配信) とコントロール群 (メールの配信なし) 購入額の平均の差に対して、t検定を実行する
data_mail = data.query("treatment == 1")
data_control = data.query("treatment == 0")
stats.ttest_ind(data_mail.spend, data_control.spend)

Ttest_indResult(statistic=5.300090294465472, pvalue=1.163200872605869e-07)

### 信頼区間

In [26]:
# サンプルサイズ
n = data_control.shape[0]
m = data_mail.shape[0]

# グループ間の差の分散
V = (((data_control.spend - data_control.spend.mean()) ** 2).sum() \
    + ((data_mail.spend - data_mail.spend.mean()) ** 2).sum()) \
     / (n + m - 2)
V

224.7512083489742

In [30]:
# グループ間の平均の差の標準誤差
SE = np.sqrt((V / m) + (V / n))
SE

0.14524793222832738

In [31]:
# 推定値
est = data_mail.spend.mean() - data_control.spend.mean()
est

0.7698271558945367

In [32]:
# 95%信頼区間
est - 1.96 * SE, est + 1.96 * SE

(0.485141208727015, 1.0545131030620583)

# 1.4.3 バイアスのあるデータによる効果の検証

## バイアスのあるデータの準備
- メールが配信されていないグループでは、以下の条件のいずれかに該当するデータをランダムに半分選んで削除
- メールが配信されるグループでは、以下の条件のいずれにも該当しないデータをランダムに半分選んで削除
    - 昨年の購入額 (history) が300より高い
    - 最後の購入からの経過時間 (recency) が3未満
    - 接触チャネル (channel) が複数ある (Multichannel)

In [62]:
data_mail_biased = data_mail.copy(deep=True)
data_mail_biased["remove"] = 0
data_mail_biased.shape

(21307, 14)

In [63]:
# ランダムに半分選ぶ
random.seed(0)
data_mail_biased.loc[data_mail_biased.query("history <= 300 and recency >= 3 and channel != 'Multichannel'").index, "remove"] = \
    data_mail_biased.query("history <= 300 and recency >= 3 and channel != 'Multichannel'").remove.apply(lambda x: random.choice([0, 1]))

In [64]:
# 半分選ばれていることを確認
data_mail_biased.query("history <= 300 and recency >= 3 and channel != 'Multichannel'").remove.mean()

0.4986746472851646

In [65]:
# 該当データを半分削除
data_mail_biased = data_mail_biased.query("remove == 0")
data_mail_biased.drop("remove", axis=1, inplace=True)
data_mail_biased.reset_index(drop=True, inplace=True)
data_mail_biased.shape

(15475, 13)

In [66]:
data_control_biased = data_control.copy(deep=True)
data_control_biased["remove"] = 0
data_control_biased.shape

(21306, 14)

In [67]:
# ランダムに半分選ぶ
random.seed(0)
data_control_biased.loc[data_control_biased.query("history > 300 or recency < 3 or channel == 'Multichannel'").index, "remove"] = \
    data_control_biased.query("history > 300 or recency < 3 or channel == 'Multichannel'").remove.apply(lambda x: random.choice([0, 1]))

In [68]:
# 半分選ばれていることを確認
data_control_biased.query("history > 300 or recency < 3 or channel == 'Multichannel'").remove.mean()

0.49824271242505686

In [69]:
# 該当データを半分削除
data_control_biased = data_control_biased.query("remove == 0")
data_control_biased.drop("remove", axis=1, inplace=True)
data_control_biased.reset_index(drop=True, inplace=True)
data_control_biased.shape

(16486, 13)

## 集計

In [70]:
# 介入群とコントロール群での、購入の発生確率と購入額の平均
data_biased = pd.concat([data_control_biased, data_mail_biased])
data_biased.groupby("treatment")[["conversion", "spend"]].mean()

,conversion,spend
treatment,,
0,0.005217,0.633393
1,0.013829,1.571542


## 有意差検定

### t検定

In [71]:
# 介入群 (男性向けメールを配信) とコントロール群 (メールの配信なし) 購入額の平均の差に対して、t検定を実行する
stats.ttest_ind(data_mail_biased.spend, data_control_biased.spend)

Ttest_indResult(statistic=5.371979697245043, pvalue=7.842046369724741e-08)

### 信頼区間

In [72]:
# サンプルサイズ
n = data_control_biased.shape[0]
m = data_mail_biased.shape[0]

# グループ間の差の分散
V = (((data_control_biased.spend - data_control_biased.spend.mean()) ** 2).sum() \
    + ((data_mail_biased.spend - data_mail_biased.spend.mean()) ** 2).sum()) \
     / (n + m - 2)
V

243.4449622599646

In [73]:
# グループ間の平均の差の標準誤差
SE = np.sqrt((V / m) + (V / n))
SE

0.1746375374930967

In [74]:
# 推定値
est = data_mail_biased.spend.mean() - data_control_biased.spend.mean()
est

0.9381493057897854

In [75]:
# 95%信頼区間
est - 1.96 * SE, est + 1.96 * SE

(0.5958597323033159, 1.2804388792762549)